In [799]:
import numpy as np
import importlib
import sklearn.datasets

from neuralnetnumpy.Optimiser import GradientDescent, AdaptiveLearningRate
from neuralnetnumpy import loss, regularisation, neuralnet, layer
from conv2dnumpy  import conv2d, pooling

import utility

#reload modules
Layer = importlib.reload(layer)
Conv2D = importlib.reload(conv2d)
Pooling = importlib.reload(pooling)
NeuralNet = importlib.reload(neuralnet)
GradientDescent = importlib.reload(GradientDescent)
AdaptiveLearningRate = importlib.reload(AdaptiveLearningRate)
Loss = importlib.reload(loss)
Regularisation = importlib.reload(regularisation)
Utility = importlib.reload(utility)

In [800]:
#optimiser
learning_rate = 0.01
optimiser_method = GradientDescent.StochasticGradientDescent(learning_rate)
#optimiser_method = GradientDescent.Momentum(learning_rate,beta=0.9)
#optimiser_method = AdaptiveLearningRate.AdaGrad(learning_rate)
#optimiser_method = AdaptiveLearningRate.RMSprop(learning_rate,beta=0.9)

In [801]:
#Regularisation
gamma = 0.05
regularisation_method = Regularisation.L2(gamma)

In [802]:
#loss
clip_grad = False
norm_grad = False
#loss_method = Loss.MeanSquared(clip_grad, norm_grad)
loss_method = Loss.BinaryCrossEntropy(clip_grad, norm_grad)

In [803]:
#hyper-parameters
epoch = 10
batch_size = 1
mode = 'train'

network_arch = [{'layer_type_category':'Convolution', 'layer_type': 'input','input_size':image.shape ,'kernel_size':(9,9), 'kernel_count':10, 'stride':9, 'padding':'valid', 'activation':'Tanh'},
                {'layer_type_category':'Pooling', 'layer_type': 'input' ,'frame_size':(9,9), 'stride':2, 'pooling_type':'average'},
                {'layer_type_category':'Convolution', 'layer_type':'hidden', 'kernel_size':(7,7), 'kernel_count':10, 'stride':7, 'padding':'valid', 'activation':'Tanh'},
                {'layer_type_category':'Pooling', 'layer_type':'hidden', 'frame_size':(2,2), 'stride':2, 'pooling_type':'average'},
                {'layer_type_category':'Dense', 'layer_type':'input', 'size':None},
                {'layer_type_category':'Dense', 'layer_type':'hidden', 'size':100,'activation':'Sigmoid'},
                {'layer_type_category':'Dense', 'layer_type':'output', 'size':10, 'activation':'Softmax'}
]                

In [804]:
class Conv2DNeuralNet(NeuralNet.NeuralNet):
    def __init__(self, network_arch, loss_method, optimiser_method, epoch, batch_size, mode,  **kwargs):
        super().__init__(network_arch, loss_method, optimiser_method, epoch, batch_size, mode, **kwargs)

    def createNetwork(self):
        for index, _layer in  enumerate(self.network_arch):
            if _layer['layer_type_category'] == 'Dense':
                if _layer.get('layer_type') != 'input':
                    if _layer.get('dropout', 0):
                        self.layers_list.append(Layer.Dropout(self.network_arch[index-1].get('size',2), _layer.get('size',2), _layer.get('activation','sigmoid'), _layer.get('layer_type_category'), _layer.get('layer_type'), _layer.get('dropout')))
                    else:
                        print('d',self.network_arch[index-1].get('size',2))
                        self.layers_list.append(Layer.Layer(self.network_arch[index-1].get('size',2), _layer.get('size',2), _layer.get('activation','sigmoid') , _layer.get('layer_type_category'), _layer.get('layer_type')))
                else:
                    self.network_arch[index]['size'] = self.activation_map_width*self.activation_map_height*self.activation_map_depth
                    print('inp',self.activation_map_width,self.activation_map_height,self.activation_map_depth)
            elif _layer['layer_type_category'] == 'Convolution':
                if _layer['layer_type'] == 'input':
                    self.activation_map_width = int((_layer.get('input_size')[0] - _layer.get('kernel_size')[0])/ _layer.get('stride')) + 1
                    self.activation_map_height = int((_layer.get('input_size')[1] - _layer.get('kernel_size')[1])/ _layer.get('stride')) + 1
                    self.activation_map_depth = _layer.get('kernel_count')
                    print(self.activation_map_width,self.activation_map_height,self.activation_map_depth)
                else:
                    self.activation_map_width = int((self.activation_map_width - _layer.get('kernel_size')[0])/ _layer.get('stride')) + 1
                    self.activation_map_height = int((self.activation_map_height - _layer.get('kernel_size')[1])/ _layer.get('stride')) + 1
                    self.activation_map_depth = _layer.get('kernel_count')
                    print('conv',self.activation_map_width,self.activation_map_height,self.activation_map_depth)
                self.layers_list.append(conv2d.Conv2D(_layer.get('kernel_size'), _layer.get('kernel_count','sigmoid') , _layer.get('stride',1), _layer.get('padding','valid'), _layer.get('activation'),_layer.get('layer_type_category'), _layer.get('layer_type')))
            elif _layer['layer_type_category'] == 'Pooling':
                self.layers_list.append(Pooling.Pooling(_layer.get('frame_size'), _layer.get('stride',1), _layer.get('layer_type_category'), _layer.get('layer_type'),  _layer.get('pooling_type')))
                self.activation_map_width = int((self.activation_map_width - _layer.get('frame_size')[0])/ _layer.get('stride')) + 1
                self.activation_map_height = int((self.activation_map_height - _layer.get('frame_size')[1])/ _layer.get('stride')) + 1
                print('pool',self.activation_map_width,self.activation_map_height,self.activation_map_depth)

    def forwardPass(self, inputs, outputs = None):
        layer_out = inputs
        drop_index = []
        for _layer in self.layers_list:
            if _layer.layer_type_category == 'Dense':
                inputs = np.ndarray.flatten(inputs)
                inputs = inputs.reshape(1,inputs.shape[0])
            print(_layer.layer_type_category)
            print(_layer.layer_type)
            print('inputs',inputs.shape)
            layer_out = _layer(inputs)
            print('layer_out',layer_out.shape)
            inputs = layer_out
            if _layer.layer_type_category != 'Pooling':
                self.regularisation.accumulateRegLoss(_layer.w)
        if self.mode.lower() == 'train':
            self.loss.getLoss(layer_out,outputs)
        elif self.mode.lower() == 'test':
            return layer_out
        
    def backProp(self, inputs):
        upstream_gradient = self.loss.loss_derivative
        for index, _layer in enumerate(reversed(self.layers_list)):
            if _layer.layer_type_category == 'Dense':
                if _layer.layer_type == 'output':
                    if _layer.activation_method.activation == 'softmax':
                        if inputs.shape[1]>1: 
                            upstream_gradient =  np.einsum('ij,ijk->ik', upstream_gradient, _layer.activation_derivative)
                        else:
                            upstream_gradient =  np.matmul(upstream_gradient, _layer.activation_derivative)
                    else:
                        upstream_gradient = np.multiply(upstream_gradient,_layer.activation_derivative)
                    upstream_gradient_w =  np.matmul(self.layers_list[len(self.layers_list)-2].y_activation.T, upstream_gradient) 
                if _layer.layer_type == 'hidden':
                    upstream_gradient =  np.matmul(upstream_gradient, self.layers_list[len(self.layers_list) -index].w.T)
                    upstream_gradient = np.multiply(upstream_gradient,_layer.activation_derivative)
                    if self.layers_list[len(self.layers_list)-index-2].layer_type_category == 'Pooling':
                        dense_inputs  = np.ndarray.flatten(self.layers_list[len(self.layers_list) -index -2].y)
                        dense_inputs = dense_inputs.reshape(1, dense_inputs.shape[0])
                        upstream_gradient_w = np.matmul(dense_inputs.T, upstream_gradient)
                    elif self.layers_list[len(self.layers_list)-index-2].layer_type_category == 'Convolution':
                        dense_inputs  = np.ndarray.flatten(self.layers_list[len(self.layers_list) -index -2].y_activation)
                        dense_inputs = dense_inputs.reshape(1, dense_inputs.shape[0])
                        upstream_gradient_w = np.matmul(dense_inputs.T, upstream_gradient)
                upstream_gradient_b = np.sum(upstream_gradient,axis=0)
                self.optimiser(_layer, upstream_gradient_w, upstream_gradient_b)
                if self.layers_list[len(self.layers_list) -index -2].layer_type_category not in ['Pooling','Convolution']:
                    weights = _layer.getWeightBias()[0]
                    if self.layers_list[len(self.layers_list) -index -2].dropout_layer:
                        drop_index = self.layers_list[len(self.layers_list) -index -2].drop_index
                        weights[drop_index,] = 0
                    self.regularisation.regLossGradient(_layer, weights)
            
            elif _layer.layer_type_category == 'Pooling':
                if _layer.pooling_type == 'average':
                    upstream_gradient =  np.matmul(upstream_gradient, self.layers_list[len(self.layers_list) -index].w.T)
                    upstream_gradient = upstream_gradient.reshape(_layer.y.shape)
                    upstream_gradient = np.multiply(upstream_gradient, _layer.y)
                    print('upstream_gradient',upstream_gradient.shape)
                    local_gradient_top_bottom = (_layer.frame_width - _layer.stride + 1)/(_layer.frame_width*_layer.frame_height)
                    local_gradient_left_right = (_layer.frame_height - _layer.stride + 1)/(_layer.frame_width*_layer.frame_height)
                    local_gradient_vertices = 1/(_layer.frame_width*_layer.frame_height)
                    local_gradient_centre = ((_layer.frame_width - _layer.stride + 1)*(_layer.frame_height - _layer.stride + 1))/(_layer.frame_width*_layer.frame_height)
                    local_gradient = np.full((self.layers_list[len(self.layers_list) -index -2].y_activation.shape),local_gradient_centre)
                    local_gradient[:,[1,-1]] = local_gradient_left_right
                    local_gradient[[1,-1],:] = local_gradient_top_bottom
                    local_gradient[0,0]=local_gradient[0,_layer.frame_width - index]=local_gradient[_layer.frame_width -1,0]=local_gradient[0,_layer.frame_height -1]=local_gradient[_layer.frame_height -1,0]=local_gradient_top_bottom
                    print('local_gradient',local_gradient.shape)
                    for row_index in range(0,)
                    upstream_gradient = np.multiply(upstream_gradient, local_gradient)
                    print('upstream_gradient2',upstream_gradient.shape)
            #elif _layer.layer_type_category == 'Convolution':
                
        for _layer_ in self.layers_list:
            w_delta, b_delta = _layer_.getLocalGradient()
            _layer_.w = _layer_.w + w_delta
            _layer_.b = _layer_.b + b_delta


In [805]:
import matplotlib.pyplot as plt

image = plt.imread('/mnt/ebs-1/rauf_bhat/git_repo_rauf/DataReduction/DSC02163.JPG', format=None)

In [806]:
np.random.seed(1)
conv = Conv2DNeuralNet(network_arch, loss_method, optimiser_method, epoch, batch_size, mode)
conv.createNetwork()

120 213 10
pool 56 103 10
conv 8 14 10
pool 4 7 10
inp 4 7 10
d 280
d 100


In [807]:
conv.forwardPass(image, np.array([1]))

Convolution
input
inputs (1080, 1920, 3)
layer_out (120, 213, 10)
Pooling
input
inputs (120, 213, 10)
layer_out (56, 103, 10)
Convolution
hidden
inputs (56, 103, 10)
layer_out (8, 14, 10)
Pooling
hidden
inputs (8, 14, 10)
layer_out (4, 7, 10)
Dense
hidden
inputs (1, 280)
layer_out (1, 100)
Dense
output
inputs (1, 100)
layer_out (1, 10)


In [808]:
conv.backProp(image)

upstream_gradient (4, 7, 10)
local_gradient (8, 14, 10)


ValueError: operands could not be broadcast together with shapes (4,7,10) (8,14,10) 

In [ ]:
row_end_index = 2
col_end_index = 2

In [ ]:
inputs[0:row_end_index,0:col_end_index,].shape

In [ ]:
np.einsum('ijk,ilk->ilj',w, inputs[0:row_end_index,0:col_end_index,])

In [ ]:
inputs=np.array([[[1,2,3],[1,0,1],[9,10,11],[5,6,7]],[[7,8,9],[11,10,2],[1,0,1],[9,10,11]],[[1,2,4],[4,50,7],[1,0,1],[9,10,11]]])
inputs[0].shape

In [ ]:
# SKlearn Digit Data Set
inputs, Y = sklearn.datasets.load_digits( n_class=10, return_X_y=True)

inputs = Utility.dataScale(inputs)


outputs = np.zeros((Y.size, Y.max()+1))
outputs[np.arange(Y.size),Y] = 1

epoch = 10
batch_size = 128
mode = 'train'
np.random.seed(1)

network_arch = [{'layer_type':'input', 'size':inputs.shape[-1]},
                {'layer_type':'hidden', 'size':10, 'activation':'Tanh'},
                {'layer_type':'output', 'size':outputs.shape[-1], 'activation':'Softmax'}]


neural_net = NeuralNet.NeuralNet(network_arch, loss_method, optimiser_method, epoch, batch_size, mode, regularisation_method = regularisation_method)
neural_net.createNetwork()
neural_net.train(inputs, outputs)
